# Análise do Feminicídio no Brasil em 2016

In [ ]:
install.packages("corrplot")
install.packages("ggpubr")

library(tidyverse)

### Leitura dos dados

In [ ]:
df1 <- read.csv("../input/dados-mortalidade/ETLSIM.DORES_2016.csv", sep="\t", stringsAsFactors=FALSE)
df2 <- read.csv("../input/dados-mortalidade/ETLSIM.DORES_2014.csv", sep="\t", stringsAsFactors=FALSE)

### Agrupamento das bases de 2016 e 2014

In [ ]:
dados_2016_2014 <- rbind(df1,df2)

### Definição das variáveis que serão utilizadas no projeto

In [ ]:
dados <- df1
# dados <- dados_2016_2014

dados_mortalidade <- dados %>% select(ano_obito, data_obito, dia_semana_obito, data_nasc, idade_obito_calculado, SEXO, def_sexo, def_raca_cor, def_est_civil, def_escol, def_loc_ocor, def_circ_obito, ocor_NOME_UF, ocor_SIGLA_UF, ocor_REGIAO, causabas_capitulo, causabas_grupo, causabas_categoria,def_gestacao, def_obito_parto, HORAOBITO)

### Criação da coluna faixa etária

In [ ]:
dados_mortalidade$faixa_etaria <- cut(dados_mortalidade$idade_obito_calculado, breaks = c(-1, 10, 20, 30, 40, 50, 60, Inf), labels = c('0-10', '10-20', '20-30','30-40', '40-50', '50-60','60+'))

head(dados_mortalidade)

In [ ]:
counts <- table(dados_mortalidade$causabas_capitulo, dados_mortalidade$faixa_etaria)
addmargins(counts)
chisq.test(counts)
barplot(counts, main="Car Distribution by Gears and VS",
  xlab="Number of Gears", col=c("orange","red", "blue", "lightblue", "purple"),
  legend = rownames(counts), beside=TRUE)

Quantidade Total de óbitos no ano 2016:

In [ ]:
nrow(dados_mortalidade)

### Tabelas com dados de agressão por sexo e outras variáveis

In [ ]:
dados_agressoes <- dados_mortalidade[(dados_mortalidade$causabas_grupo=="Agressões"),]

#### Sexo x Ano óbito

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$ano_obito))

#### Sexo x Dia da Semana

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$dia_semana_obito))

#### Sexo x Faixa Etária

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$faixa_etaria))

#### Sexo x Estado Civil

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$def_est_civil))

#### Sexo x Local do óbito

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$def_loc_ocor))

#### Sexo x Horário

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$HORAOBITO))

#### UF x Sexo

In [ ]:
addmargins(table(dados_agressoes$ocor_SIGLA_UF, dados_agressoes$def_sexo))

#### Cor x Sexo

In [ ]:
addmargins(table(dados_agressoes$def_raca_cor, dados_agressoes$def_sexo))

#### Categoria x Sexo

In [ ]:
addmargins(table(dados_agressoes$causabas_categoria, dados_agressoes$def_sexo))

### Formatação da hora do óbito

In [ ]:
dados_agressoes$HORAOBITO <- sprintf("%04d", dados_agressoes$HORAOBITO)
dados_agressoes

x <- gsub("(.{2})", "\\1:", dados_agressoes$HORAOBITO)
dados_agressoes$hora_format <- substr(x, 0, 5)
dados_agressoes$hora <- substr(x, 0, 2)

In [ ]:
agressoes_hora <- table(dados_agressoes$hora, dados_agressoes$def_sexo)
addmargins(agressoes_hora)

# write.csv(agressoes_hora,"hora.csv", row.names = TRUE)

### Tabela de categoria por causas

In [ ]:
addmargins(table(dados_agressoes$causabas_categoria, dados_agressoes$causabas_grupo))

### Teste de Qui-Quadrado com dia da semana e hora do óbito

In [ ]:
dados_filtrados <- dados_agressoes[!(dados_agressoes$hora=="  "),]

t <- table(dados_filtrados$dia_semana_obito, dados_filtrados$hora)

addmargins(t)
addmargins(prop.table(t))

chisq.test(t)

### Teste de Qui-Quadrado com a categoria e local de ocorrência do óbito

In [ ]:
table3 <- table(dados_agressoes$causabas_categoria, dados_agressoes$def_loc_ocor)

addmargins(table3)
addmargins(prop.table(table3))

chisq.test(table3)

## Comparações com o ano de 2014

In [ ]:
addmargins(table(dados_agressoes$def_sexo, dados_agressoes$ano_obito))

In [ ]:
dados_mulheres <- dados_agressoes[(dados_agressoes$SEXO==2),]
addmargins(table(dados_agressoes$def_loc_ocor, dados_agressoes$def_sexo))

In [ ]:
dados_populacao <- read.csv("../input/dadospopulacao/hora_format - Folha3.csv", sep=",", stringsAsFactors=FALSE)

dados_populacao$ocor_NOME_UF <- str_to_title(dados_populacao$ocor_NOME_UF)
dados_mulheres$ocor_NOME_UF <- str_to_title(dados_mulheres$ocor_NOME_UF)

dados_mulheres <- merge(dados_populacao, dados_mulheres)

### Dados de jogos de futebol do Flamengo

In [ ]:
dados_flamengo <- read.csv("../input/dadosflamengo/dados_flamengo.csv", sep=",", stringsAsFactors=FALSE)

dados_mulheres$Data <- substr(dados_mulheres$data_obito, 0, 10)
dados_mulheres

dados_merge <- merge(dados_mulheres, dados_flamengo)
dados_merge